### SGD Implementation Example

Test the new data then train on that new data

In [9]:
import numpy as np
from sklearn import linear_model
from sklearn import metrics

In [10]:
n_samples, n_features = 10, 5
np.random.seed(0)
y = np.random.randn(n_samples)
X = np.random.randn(n_samples, n_features)

In [30]:
clf = linear_model.SGDRegressor()
rmseList = list()
for instanceIndex in range(0, len(X)):
    instance = X[instanceIndex].reshape(1, -1)
    label = y[instanceIndex].reshape(-1, 1)
    if instanceIndex == 0:
        #train the model
        clf.fit(instance,label)
    else:
        #test the model
        predicted = clf.predict(instance)
        #get rmse
        rmse = np.sqrt(metrics.mean_squared_error(label, predicted))
        rmseList.append(rmse)

/home/sof/.local/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDRegressor in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)
/home/sof/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [31]:
rmseList

[0.22059874602133628,
 0.7705616994897968,
 2.138493820098493,
 1.834517235547889,
 1.211007783645852,
 1.1151303563431092,
 0.1128506255106105,
 0.16181025119858136,
 0.3347571060085341]

### SGD Implementation

In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.model_selection import train_test_split
import DataPipeline
from sklearn import linear_model
import pickle
from math import trunc
import copy

In [2]:
def createRandomisedVOCScreenings(vocRandomised, runtimeList, movieList ,matchedMovies):
    startIndex = 0
    screeningList = list()
    for movie in matchedMovies:
        try:
            runtime = runtimeList[movieList.index(movie)]
        except ValueError:
            continue
        endIndex = startIndex + runtime
        screening = vocRandomised[startIndex:endIndex]
        screeningList.append(screening)
        startIndex = endIndex
    return screeningList

In [3]:
def generateVOCScreenings(vocDf2013,vocDf2015, sliceDf, matchedMovies):
    screeningList = list()
    prevStartIndex = 0
    startIndex = 0
    vocDf = vocDf2013
    for index in range(0, len(matchedMovies)):
        
        startIndex = sliceDf.loc[index]['start']
        endIndex = sliceDf.loc[index]['end']
        if startIndex == 371: #the 2015 df starts at this index
            vocDf = vocDf2015
        screening = pd.DataFrame(vocDf.iloc[startIndex:endIndex+1,0])
        screeningList.append(screening)
        
        prevStartIndex = startIndex
        
    return screeningList

In [4]:
def normalisation(vocScreenings, voc):
    normalisedVOCList = list()
    for screening in vocScreenings:
        normalisedVOCFrame = copy.deepcopy(screening)
        normalisedVOCFrame = normalisedVOCFrame.values/max(screening.values)
        normalisedVOCFrame = normalisedVOCFrame.flatten()
        normalisedScreening= pd.DataFrame.from_dict({voc:normalisedVOCFrame})
        normalisedVOCList.append(normalisedScreening)
    return normalisedVOCList

In [5]:
#some vocs have NaN measurements during the decided screening times. Ignore these screenings
#also remove empty screenings
def removeNaNScreenings(screenings, randomisedScreenings, matchedMovies):
    screeningList = list()
    randomScreeningList = list()
    movieList = list()
    for screeningIndex in range(0, len(screenings)):
        if not(np.isnan(screenings[screeningIndex].values).any()) and len(screenings[screeningIndex].values) != 0:
            screeningList.append(screenings[screeningIndex])
            randomScreeningList.append(randomisedScreenings[screeningIndex])
            movieList.append(matchedMovies[screeningIndex])
    return screeningList,randomScreeningList,movieList

In [6]:
#column header matching issue between 2013 and 2015 
#e.g. in 2015 column is m356.0711 vs in 2013 is it m356.0714
#assumption being made is that they are the same column so round to 2dp and match
def vocRounding(vocDf):
    vocList = list()
    for index in range(0, len(vocDf.columns)):
        if vocDf.columns[index] == 'Time' or vocDf.columns[index] == 'ocs' or vocDf.columns[index] == 'co' or vocDf.columns[index] == 'CO2':
            vocList.append(vocDf.columns[index])    
        else:
            #string slice to get the molar mass
            voc = vocDf.columns[index]
            mass = (trunc(float(voc[1:])*1000))/1000 #TRUNCATE TO 3DP
            vocList.append(mass)
    return vocList

In [7]:
#frames were collected at 1/3fps so for a 30 second period there are 10 frames. This function just groups the 
#dominant frame colour or shade components to within their respective intervals
def grouping(visualList):
    movieVisuals = list()
    for index in range(0, int(len(visualList)/10)):
        segment = visualList[index*10:index*10+10]
        movieVisuals.append(segment)
    return movieVisuals

In [8]:
def processVisuals(movieVisualData, runtime, isColour):
    visualDataIntervals = grouping(movieVisualData)
    #the visual data also has the credits accounted for so remove them
    visualDataIntervals = visualDataIntervals[:runtime]
    #create a dataframe 
    if isColour: 
        #create a dominant colour dataframe
        framesPerInterval = 10
        header = list();
        for i in range(1,framesPerInterval+1):
            header = header + ['R'+str(i), 'G' + str(i),  'B'+str(i)]
    else: #shade object to be parsed
        framesPerInterval = 10
        header = ['S' + str(x) for x in range(1,framesPerInterval+1)]
    
    visualDf = pd.DataFrame(columns=header)
    #assemble the dataframe
    for segment in visualDataIntervals:
        index = visualDataIntervals.index(segment)
        colourRow = list()
        for colour in segment:
            if isColour:
                colourRow = colourRow + [colour[0], colour[1], colour[2]]
            else:
                colourRow = colourRow + [colour[0]]
        #assign that colour row to the dataframe
        visualDf.loc[index] = colourRow
            
    return visualDf

In [9]:
def processAudio(runtime, audio):
    audioFeatures = list(audio.keys())

    audioDf = pd.DataFrame(columns=[])        
    for key in audioFeatures:
        audio[key] = audio[key][:runtime]

        #assemble df 
        #create header
        if key != 'tempo':
            header = [key + str(x) for x in range(1, len(audio[key][0])+1)]
        else:
            header = ['tempo']

        audioFeatureDf = pd.DataFrame(columns=header)
        for index in range(0, len(audio[key])):
            feature = audio[key][index]
            audioFeatureDf.loc[index] = feature

        #concatenate featureDf to audioDf
        audioDf = pd.concat([audioDf,audioFeatureDf], axis=1)
    
    return audioDf

In [10]:
def processSubtitles(subs, effectiveRuntime):
    
    header = ['sentiment value']
    subSentimentDf = pd.DataFrame(columns=header)
    for sentimentIndex in range(0, len(subs)):
        sentiment = subs[sentimentIndex]
        if len(sentiment) != 0:
            if sentiment['sentimentValue'] == np.NaN:
                print('YES')
            else:         
                subSentimentDf.loc[sentimentIndex] = [sentiment['sentimentValue']]
        else:
            subSentimentDf.loc[sentimentIndex] = [-1] #indicates no dialog occurred during the scene
        
        #enforce no dialog until the credit scene if there is in fact no dialog
        if len(subSentimentDf) != effectiveRuntime:
            #no dialog at the end thus need to fill the rest with -1
            for index in range(0, effectiveRuntime-len(subSentimentDf)+1):
                 subSentimentDf.loc[index] = [-1]
    
    return subSentimentDf

In [11]:
def processASL(asl, effectiveRuntime):
    
    header = ['average shot length']
    aslDf = pd.DataFrame(columns=header)
    for index in range(0, effectiveRuntime): 
        aslValue = asl[index]
        aslDf.loc[index] = aslValue
    
    return aslDf

In [12]:
def removeMovies(vocDict):

    #remove all screenings of im off then and help i shrunk the teacher as at the current time do not have the movies
    screenings = list()
    matchedMovies = list()
    for movieIndex in range(0, len(vocDict['matchedMovies'])):
        movie = vocDict['matchedMovies'][movieIndex]
        if movie != "Help, I Shrunk My Teacher" and movie != "I'm Off Then":
            #add good screenings to a modified screening list
            matchedMovies.append(movie)
            screenings.append(vocDict['screenings'][movieIndex])
    #replace
    vocDict = dict()
    vocDict['matchedMovies'] = matchedMovies
    vocDict['screenings'] = screenings
    
    return vocDict

In [13]:
#user macros
featureDf = pd.DataFrame([]) #film feature dataframe
labelDf = pd.DataFrame([]) #voc dataframe
deltaVOCs = False
windowedVOCs = False
lengthOfWindow = 10

In [14]:
#read in the various csvs
#2013 Dataset
vocPath = 'Numerical Data/2013VOCData.csv'
voc2013DfAll = pd.read_csv(vocPath, header = 0, nrows = 74208, low_memory=False)
movieScreeningsPath = 'Numerical Data/screening_times.csv'
movingScreeningsDf = pd.read_csv(movieScreeningsPath, usecols = ['scheduled','movie','filled %'])
movieRuntimesPath = 'Numerical Data/movie_runtimes.csv'
movieRuntimeDf = pd.read_csv(movieRuntimesPath, usecols = ['movie', 'runtime (mins)', 'effective runtime'])
#2015 Dataset
starWarsPath = 'Numerical Data/Star Wars-The Force Awakens.csv'
starWarsScreeningDf = pd.read_csv(starWarsPath)
imOffThenPath = 'Numerical Data/I\'m Off Then.csv'
imOffThenScreeningDf = pd.read_csv(imOffThenPath)
helpIShrunkTheTeacherPath = 'Numerical Data/Help, I Shrunk My Teacher.csv'
helpIShrunkTheTeacherScreeningDf = pd.read_csv(helpIShrunkTheTeacherPath)
vocPath = 'Numerical Data/2015VOCData.csv'
voc2015DfAll = pd.read_csv(vocPath)
#remove first column of 2015 voc df as its not used
voc2015DfAll.drop("Unnamed: 0", axis=1, inplace=True)

#import co2Slice pickle objects
slicePath = 'Pickle Objects/CO2SliceDict.p'
sliceDict = pickle.load(open(slicePath, "rb" )) #contains df of co2 slice indices and matched movie list

In [15]:
#overall feature and labels df
featureDf = pd.DataFrame([]) #film feature dataframe
labelDf = pd.DataFrame([]) #voc dataframe

#import movie runtimes
movieRuntimesPath = 'Numerical Data/movie_runtimes.csv'
movieRuntimeDf = pd.read_csv(movieRuntimesPath, usecols = ['movie', 'runtime (mins)', 'effective runtime'])
movieList = list(movieRuntimeDf['movie'])

movieFeatureDict = dict() #dict contains the movie film features with the keys being the movies
#import pickle objects for movies and then assemble the dataframes  
for movie in movieList:
    try:
        #load pickle feauture objects
        featurePath = 'Pickle Objects/Audio Feature Pickle Objects/' + movie + '.p'
        audio = pickle.load(open(featurePath, "rb" )) 
        featurePath = 'Pickle Objects/Colour Pickle Objects/' + movie + '.p'
        colour = pickle.load(open(featurePath, "rb" )) 
        featurePath = 'Pickle Objects/Shade Pickle Objects/' + movie + '.p'
        shade = pickle.load(open(featurePath, "rb" )) 
        featurePath = 'Pickle Objects/Subtitle Sentiment Pickle Objects/' + movie + '.p'
        sentiment = pickle.load(open(featurePath, "rb" )) 
        featurePath = 'Pickle Objects/ASL Pickle Objects/' + movie + '.p'
        asl = pickle.load(open(featurePath, "rb" )) 

        runtime = int(movieRuntimeDf.loc[movieList.index(movie)]['effective runtime'])
        colourDf = processVisuals(colour, runtime, True)
        shadeDf = processVisuals(shade, runtime, False)
        audioDf = processAudio(runtime, audio)
        sentimentDf = processSubtitles(sentiment,runtime)
        aslDf = processASL(asl, runtime)

        inputDf = pd.concat([colourDf,shadeDf,audioDf,sentimentDf,aslDf], axis = 1)
        movieFeatureDict[movie] = inputDf

    except FileNotFoundError:
        pass

In [16]:
voc2015Col = vocRounding(voc2015DfAll)
voc2013Col = vocRounding(voc2013DfAll)
voc2013Df = copy.deepcopy(voc2013DfAll)
voc2015Df = copy.deepcopy(voc2015DfAll)
voc2013Df.columns = voc2013Col
voc2015Df.columns = voc2015Col

vocUseList = list()

for vocIndex in range(0, len(voc2015Df.columns)):
    voc = voc2015Df.columns[vocIndex]
    if voc == 'Time':
        continue
    else:
        try:
            indexMask = list(voc2013Df.columns).index(voc)
        except ValueError: #the voc isnt within the 2013 VOC dataset
            continue 
            
        print(voc)
        #create normal voc screening list
        vocDf2013 = voc2013Df.iloc[:,[indexMask]]
        vocDf2015 = voc2015Df.iloc[:,[vocIndex]]   
        
        #generate screenings
        screeningList = generateVOCScreenings(vocDf2013,vocDf2015, sliceDict['sliceDf'], sliceDict['matchedMovies'])
        matchedMovies = copy.deepcopy(sliceDict['matchedMovies'])
        #remove normal screenings with NaN values in the screenings
        screeningList, randomisedScreeningList, matchedMovies = removeNaNScreenings(screeningList, screeningList, matchedMovies)
        #normalise both screenings 
        screeningList = normalisation(screeningList, voc)
        #create randomised and unrandomised list
        vocDict = {'screenings':screeningList, 'matchedMovies':matchedMovies}
        #remove all screenings of im off then and help i shrunk the teacher as at the current time do not have the movies
        vocDict = removeMovies(vocDict)
        
        #create overall label and feature df
        for i in range(0, len(vocDict['screenings'])): 

            matchedMovie = vocDict['matchedMovies'][i]

            if not(deltaVOCs):
                featureDf = pd.concat([featureDf, movieFeatureDict[matchedMovie]])

            if not(windowedVOCs):
                screening = vocDict['screenings'][i]
                labelDf = pd.concat([labelDf, screening['CO2']])

        #relabel column title 
        if not(windowedVOCs):
            labelDf.columns = ['VOC']

        
        break

CO2


In [17]:
print(featureDf.shape[0])
print(labelDf.shape)

21085
(21085, 1)


In [37]:
initialBatch = 100
clf = linear_model.SGDRegressor()
rmseList = list()

#run the initial batch training
instance = featureDf.iloc[:initialBatch].values
label = labelDf.iloc[:initialBatch].values
clf.fit(instance,label)


for instanceIndex in range(initialBatch+1, labelDf.shape[0]):
    instance = (featureDf.iloc[instanceIndex].values).reshape(1, -1)
    label = (labelDf.iloc[instanceIndex].values).reshape(-1, 1)
    #test the model
    predicted = clf.predict(instance)
    print(predicted)
    #get rmse
    rmse = np.sqrt(metrics.mean_squared_error(label, predicted))
    rmseList.append(rmse)

/home/sof/.local/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDRegressor in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)
/home/sof/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[4.43803504e+15]
[3.06188879e+15]
[1.95934234e+15]
[4.08157504e+15]
[3.85077643e+15]
[2.4442326e+15]
[4.10995008e+15]
[3.8998194e+15]
[1.22179185e+15]
[1.51786338e+15]
[1.50639098e+15]
[6.43196971e+15]
[3.84823945e+15]
[4.32501053e+15]
[1.79682573e+15]
[5.12859208e+15]
[4.75668883e+15]
[3.44932792e+15]
[3.80738723e+15]
[3.83500882e+15]
[5.05946442e+15]
[5.22163649e+15]
[3.55876118e+15]
[4.85573239e+15]
[7.06540672e+15]
[5.71102397e+15]
[8.28428991e+15]
[7.19759932e+15]
[3.50084998e+15]
[6.92015499e+15]
[2.87191662e+15]
[3.32143279e+15]
[3.20335484e+15]
[1.73987718e+15]
[1.23510387e+15]
[2.55802913e+15]
[4.08651286e+15]
[3.73781871e+15]
[5.94769941e+15]
[5.30486921e+15]
[3.73288749e+15]
[4.76909461e+15]
[4.55987142e+15]
[4.48763642e+15]
[5.06735228e+15]
[9.52527007e+14]
[2.2031085e+15]
[6.01708461e+15]
[4.10391783e+15]
[3.98215448e+15]
[3.33711673e+15]
[3.15104517e+15]
[4.29264757e+15]
[5.99401676e+15]
[3.80305038e+15]
[2.36321526e+15]
[4.91234433e+15]
[5.94980209e+15]
[4.35663572e+15]


[4.16348049e+15]
[4.76107858e+15]
[3.1081518e+15]
[3.94737403e+15]
[2.9648862e+15]
[3.26798021e+15]
[3.85671774e+15]
[2.62863475e+15]
[3.35205134e+15]
[3.22780691e+15]
[2.96807401e+15]
[4.68962381e+15]
[2.42272361e+15]
[2.52787115e+15]
[-1.13440941e+15]
[3.15118293e+15]
[5.00297214e+15]
[4.97446757e+15]
[3.85173957e+15]
[2.60961882e+15]
[5.45906492e+15]
[5.97987232e+15]
[5.10823669e+15]
[4.08113184e+15]
[4.83453633e+15]
[3.42024786e+15]
[1.54106655e+15]
[4.29134804e+15]
[6.39194072e+15]
[3.58418957e+15]
[2.21676101e+15]
[4.08393281e+15]
[5.48058858e+15]
[6.42673446e+15]
[2.90927301e+15]
[5.45151591e+15]
[6.25958227e+15]
[3.49834625e+15]
[1.50249431e+15]
[3.63342287e+15]
[4.95098289e+15]
[4.97394011e+15]
[5.54444214e+15]
[6.75786675e+15]
[5.95874746e+15]
[6.09977274e+15]
[4.81156244e+15]
[4.80246014e+15]
[3.17622887e+15]
[2.78889338e+15]
[2.86857721e+15]
[2.20776165e+15]
[5.58565156e+15]
[4.30936813e+15]
[4.39061116e+15]
[3.43216621e+15]
[3.92867343e+15]
[3.8565826e+15]
[4.8598357e+15]


[7.15662215e+15]
[5.71931429e+15]
[2.99370098e+15]
[3.74805387e+15]
[5.31113092e+15]
[5.22506737e+15]
[4.29595772e+15]
[6.29432523e+15]
[9.6341942e+15]
[3.48955986e+15]
[2.23248443e+15]
[2.06970544e+15]
[3.5090812e+15]
[1.88317839e+15]
[8.00575162e+15]
[6.66373665e+15]
[3.79949365e+15]
[3.21544968e+15]
[6.48685335e+15]
[4.97206635e+15]
[5.426696e+15]
[6.14418831e+15]
[5.90134905e+15]
[7.31759759e+15]
[4.30828834e+15]
[5.85874504e+15]
[4.69015744e+15]
[5.2552827e+15]
[2.38367287e+15]
[2.92964527e+15]
[1.83724683e+15]
[4.65787376e+15]
[1.96400095e+15]
[1.99758853e+15]
[3.29968999e+15]
[3.14332005e+15]
[2.83579369e+15]
[4.39221513e+15]
[3.78054774e+15]
[3.23977339e+15]
[2.88072159e+15]
[3.09814626e+15]
[2.24716719e+15]
[4.0959821e+15]
[1.75846039e+15]
[2.76660014e+15]
[2.44431013e+15]
[4.03306301e+15]
[3.50214872e+15]
[4.38002656e+15]
[1.85552399e+15]
[6.42319662e+15]
[5.27126324e+15]
[2.78258448e+15]
[4.71000507e+15]
[4.64102384e+15]
[5.5948372e+15]
[2.51651546e+15]
[5.83593457e+15]
[3.4

[2.0780776e+15]
[2.22789988e+15]
[4.45439401e+15]
[4.70299295e+15]
[2.89971615e+15]
[2.88822083e+15]
[3.84535009e+15]
[4.40896712e+15]
[1.68249332e+15]
[2.30175075e+15]
[8.83389191e+15]
[9.30909107e+15]
[-1.40438278e+14]
[3.48336871e+15]
[4.79396581e+15]
[8.4606927e+15]
[7.44230445e+15]
[8.79591768e+15]
[4.04292988e+15]
[3.72882024e+15]
[2.87698899e+15]
[3.08240547e+15]
[2.83568091e+15]
[4.03357402e+15]
[5.42082234e+15]
[6.79320932e+15]
[5.75739629e+15]
[1.77120628e+15]
[2.67393416e+15]
[2.92734378e+15]
[2.81615947e+15]
[2.13458588e+15]
[6.5116388e+15]
[5.0047856e+15]
[5.7721801e+15]
[5.50097123e+15]
[5.61461752e+15]
[7.86645868e+15]
[3.97052636e+15]
[6.20490486e+15]
[5.83679235e+15]
[5.81135491e+15]
[5.65324341e+15]
[3.48730834e+15]
[3.43443248e+15]
[5.73651625e+15]
[4.1363788e+15]
[5.05280022e+15]
[4.11949923e+15]
[1.35565495e+15]
[2.61738813e+15]
[1.74641432e+15]
[2.97656479e+15]
[3.62017131e+15]
[3.24264829e+15]
[4.8964593e+14]
[1.22036169e+15]
[1.09899311e+15]
[2.17948434e+15]
[5.

...

[1.75582919e+15]
[5.31123142e+15]
[3.70966449e+15]
[4.88176724e+15]
[5.41741355e+15]
[4.5552756e+15]
[3.76645641e+15]
[4.22126492e+15]
[1.358855e+15]
[-6.73491569e+14]
[6.76076145e+13]
[3.49370942e+15]
[1.409231e+15]
[3.73795311e+15]
[3.82261979e+15]
[5.58360669e+15]
[6.65761832e+15]
[6.57573579e+15]
[6.23303553e+15]
[5.75675091e+15]
[3.74522781e+15]
[7.27759211e+15]
[8.37117764e+15]
[6.13970528e+15]
[3.71842699e+15]
[4.95901008e+15]
[8.70836893e+14]
[5.32850557e+14]
[1.20687492e+15]
[7.52392583e+15]
[4.09760509e+15]
[4.51904133e+15]
[5.02571854e+15]
[3.01857534e+15]
[3.40360038e+15]
[5.31177905e+15]
[4.02487674e+15]
[5.31794955e+15]
[3.58443016e+15]
[2.50880908e+15]
[4.86468918e+15]
[3.55389691e+15]
[3.38145057e+15]
[5.44688749e+15]
[2.97253063e+15]
[3.17972299e+15]
[6.6250755e+15]
[-2.98703848e+14]
[2.26990477e+15]
[5.96452342e+15]
[7.47145833e+15]
[9.02048978e+15]
[7.66175511e+15]
[1.89705215e+15]
[6.22767875e+15]
[3.01882018e+15]
[5.67057607e+15]
[5.61519361e+15]
[3.89787246e+15]
[

[3.56470323e+15]
[2.86359152e+15]
[5.85057475e+14]
[1.27024242e+15]
[4.78663286e+15]
[6.91135062e+15]
[6.41962389e+15]
[5.13948152e+15]
[4.14921881e+15]
[4.77911638e+15]
[4.65226525e+15]
[3.19961605e+15]
[2.996532e+14]
[7.95689059e+14]
[-3.57456517e+14]
[4.82967608e+15]
[6.03647875e+15]
[7.02236242e+15]
[2.84751997e+15]
[4.61354546e+15]
[4.44225977e+15]
[4.11130621e+15]
[5.72271924e+15]
[1.41539863e+15]
[1.08193298e+15]
[3.48125981e+15]
[5.48959606e+15]
[4.41778769e+15]
[4.81920146e+15]
[5.76500299e+15]
[6.97584481e+15]
[3.50578703e+15]
[6.44186612e+15]
[6.37441117e+15]
[6.21744485e+15]
[4.22189349e+15]
[3.68363722e+15]
[3.41102162e+15]
[2.88277237e+15]
[-1.35642625e+15]
[3.30606637e+14]
[5.23323544e+15]
[2.50536684e+15]
[4.087669e+15]
[2.39331318e+15]
[3.26959815e+15]
[3.53701608e+15]
[4.75368579e+15]
[4.45293035e+15]
[4.63158555e+15]
[4.20414779e+15]
[4.46242251e+15]
[3.59764262e+15]
[4.21568178e+15]
[1.42321489e+15]
[1.29060604e+15]
[7.23730676e+14]
[-1.36829519e+15]
[1.81840826e+14

[7.63448999e+15]
[4.32501227e+15]
[2.16410591e+15]
[1.85671526e+15]
[3.07451143e+15]
[2.59531759e+15]
[3.36499582e+15]
[2.82247495e+15]
[2.19255818e+15]
[2.93907154e+15]
[5.56334119e+14]
[4.92343272e+15]
[2.9585153e+15]
[-1.19183615e+15]
[-1.40848353e+15]
[2.61100613e+15]
[6.92745198e+15]
[6.73209354e+15]
[2.70472372e+15]
[3.63308033e+15]
[3.64038309e+15]
[5.5949006e+15]
[4.0694207e+15]
[4.37606226e+15]
[6.95380325e+15]
[5.82782344e+15]
[4.01294842e+15]
[2.83648843e+15]
[3.4892679e+15]
[4.73912287e+15]
[4.85440763e+15]
[5.26393482e+15]
[4.37846954e+15]
[2.0183343e+15]
[4.68835945e+15]
[4.83722165e+15]
[2.03878358e+15]
[6.84406938e+15]
[3.91802598e+15]
[1.75582919e+15]
[5.31123142e+15]
[3.70966449e+15]
[4.88176724e+15]
[5.41741355e+15]
[4.5552756e+15]
[3.76645641e+15]
[4.22126492e+15]
[1.358855e+15]
[-6.73491569e+14]
[6.76076145e+13]
[3.49370942e+15]
[1.409231e+15]
[3.73795311e+15]
[3.82261979e+15]
[5.58360669e+15]
[6.65761832e+15]
[6.57573579e+15]
[6.23303553e+15]
[5.75675091e+15]
[3.7

[5.72918645e+15]
[5.11650895e+15]
[3.06936587e+15]
[5.00329792e+15]
[4.51618025e+15]
[3.56470323e+15]
[2.86359152e+15]
[5.85057475e+14]
[1.27024242e+15]
[4.78663286e+15]
[6.91135062e+15]
[6.41962389e+15]
[5.13948152e+15]
[4.14921881e+15]
[4.77911638e+15]
[4.65226525e+15]
[3.19961605e+15]
[2.996532e+14]
[7.95689059e+14]
[-3.57456517e+14]
[4.82967608e+15]
[6.03647875e+15]
[7.02236242e+15]
[2.84751997e+15]
[4.61354546e+15]
[4.44225977e+15]
[4.11130621e+15]
[5.72271924e+15]
[1.41539863e+15]
[1.08193298e+15]
[3.48125981e+15]
[5.48959606e+15]
[4.41778769e+15]
[4.81920146e+15]
[5.76500299e+15]
[6.97584481e+15]
[3.50578703e+15]
[6.44186612e+15]
[6.37441117e+15]
[6.21744485e+15]
[4.22189349e+15]
[3.68363722e+15]
[3.41102162e+15]
[2.88277237e+15]
[-1.35642625e+15]
[3.30606637e+14]
[5.23323544e+15]
[2.50536684e+15]
[4.087669e+15]
[2.39331318e+15]
[3.26959815e+15]
[3.53701608e+15]
[4.75368579e+15]
[4.45293035e+15]
[4.63158555e+15]
[4.20414779e+15]
[4.46242251e+15]
[3.59764262e+15]
[4.21568178e+15]

[7.4211891e+15]
[2.90249228e+15]
[4.54210801e+15]
[1.42914148e+15]
[3.3446881e+15]
[4.39047144e+15]
[3.05059704e+15]
[2.80562202e+15]
[7.63448999e+15]
[4.32501227e+15]
[2.16410591e+15]
[1.85671526e+15]
[3.07451143e+15]
[2.59531759e+15]
[3.36499582e+15]
[2.82247495e+15]
[2.19255818e+15]
[2.93907154e+15]
[5.56334119e+14]
[4.92343272e+15]
[2.9585153e+15]
[-1.19183615e+15]
[-1.40848353e+15]
[2.61100613e+15]
[6.92745198e+15]
[6.73209354e+15]
[2.70472372e+15]
[3.63308033e+15]
[3.64038309e+15]
[5.5949006e+15]
[4.0694207e+15]
[4.37606226e+15]
[6.95380325e+15]
[5.82782344e+15]
[4.01294842e+15]
[2.83648843e+15]
[3.4892679e+15]
[4.73912287e+15]
[4.85440763e+15]
[5.26393482e+15]
[4.37846954e+15]
[2.0183343e+15]
[4.68835945e+15]
[4.83722165e+15]
[2.03878358e+15]
[6.84406938e+15]
[3.91802598e+15]
[1.75582919e+15]
[5.31123142e+15]
[3.70966449e+15]
[4.88176724e+15]
[5.41741355e+15]
[4.5552756e+15]
[3.76645641e+15]
[4.22126492e+15]
[1.358855e+15]
[-6.73491569e+14]
[6.76076145e+13]
[3.49370942e+15]
[1.4

In [94]:
resultsDf = pd.DataFrame(rmseList,columns=['RMSE'])
resultsPath = str(voc) + '_SGD.csv'
resultsDf.to_csv(resultsPath, sep=',', encoding='utf-8')